In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import spikeinterface.full as si
import spikeinterface.sortingcomponents.peak_detection as pkd
import numpy as np
import time

c:\Users\aurel\Documents\Travail\Rust\Rust-Spike-Sorting\spikeinterface_rust\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import logging
FORMAT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(logging.INFO)

In [ ]:
rec, sorting = si.generate_ground_truth_recording(durations=[60],num_channels=128,sampling_frequency=25000.0,num_units= 100, seed=42)
rec = rec.save_to_memory()
rec.noise_levels = si.get_noise_levels(rec, random_slices_kwargs=dict(num_chunks_per_segment=20, chunk_size=1000))

noise_level (no parallelization): 100%|██████████| 20/20 [00:00<00:00, 187.20it/s]


In [9]:
job_kwargs={"n_jobs": 1, "mp_context": "spawn", "pool_engine":"thread", "progress_bar": True, "chunk_duration":"1s"}

In [13]:
n = 1
import cProfile, pstats, io
from pstats import SortKey
pr = cProfile.Profile()
pr.enable()

peaks = pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "numba"}, job_kwargs=job_kwargs)

pr.disable()
s = io.StringIO()
sortby = SortKey.TIME
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

noise_level (no parallelization): 100%|██████████| 20/20 [00:02<00:00,  9.80it/s]


Compute noise levels time: 2.044 s


detect peaks (locally_exclusive) (no parallelization): 100%|██████████| 300/300 [00:15<00:00, 20.00it/s]

Detect peaks 'locally_exclusive' total time: 17.061 s
         167121 function calls (164983 primitive calls) in 17.069 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      300    7.767    0.026   11.127    0.037 C:\Users\aurel\Documents\Travail\Rust\Rust-Spike-Sorting\spikeinterface\src\spikeinterface\sortingcomponents\peak_detection\locally_exclusive.py:131(detect_peaks_numba_locally_exclusive_on_chunk)
      300    3.344    0.011    3.344    0.011 {method 'nonzero' of 'numpy.ndarray' objects}
      640    2.173    0.003   34.564    0.054 C:\Users\aurel\AppData\Local\Programs\Python\Python313\Lib\asyncio\base_events.py:1970(_run_once)
      640    1.471    0.002    1.551    0.002 {built-in method select.select}
       40    1.404    0.035    1.404    0.035 {method 'partition' of 'numpy.ndarray' objects}
       41    0.193    0.005    0.193    0.005 {method 'copy' of 'numpy.ndarray' objects}
       20    0.142    0.007   

In [14]:
S_rust=0
S_numba=0

for _ in range(n):
    
    start = time.time()
    peaks = pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "numba"}, job_kwargs=job_kwargs)
    S_numba += time.time() - start
    start = time.time()
    peaks = pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "rust"}, job_kwargs=job_kwargs)
    S_rust += time.time() - start
print(f"Average time over {n} runs for locally_exclusive with rust engine: {S_rust/n:.2f} s")
print(f"Average time over {n} runs for locally_exclusive with numba engine: {S_numba/n:.2f} s")

Compute noise levels time: 0.000 s


detect peaks (locally_exclusive) (no parallelization): 100%|██████████| 300/300 [00:09<00:00, 32.12it/s]


Detect peaks 'locally_exclusive' total time: 9.354 s
Compute noise levels time: 0.000 s


detect peaks (locally_exclusive) (no parallelization): 100%|██████████| 300/300 [00:10<00:00, 27.48it/s]

Detect peaks 'locally_exclusive' total time: 10.935 s
Average time over 1 runs for locally_exclusive with rust engine: 10.94 s
Average time over 1 runs for locally_exclusive with numba engine: 9.35 s


In [7]:
%timeit pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "numba"}, job_kwargs={"n_jobs": -1, "mp_context": "spawn", "pool_engine":"thread", "progress_bar": False, "chunk_duration":"1s"})

Compute noise levels time: 0.000 s
Detect peaks 'locally_exclusive' total time: 13.770 s
Compute noise levels time: 0.000 s
Detect peaks 'locally_exclusive' total time: 14.365 s
Compute noise levels time: 0.000 s
Detect peaks 'locally_exclusive' total time: 13.616 s
Compute noise levels time: 0.000 s
Detect peaks 'locally_exclusive' total time: 13.411 s
Compute noise levels time: 0.000 s
Detect peaks 'locally_exclusive' total time: 12.938 s
Compute noise levels time: 0.000 s
Detect peaks 'locally_exclusive' total time: 14.001 s
Compute noise levels time: 0.000 s
Detect peaks 'locally_exclusive' total time: 12.980 s
Compute noise levels time: 0.000 s
Detect peaks 'locally_exclusive' total time: 12.462 s
13.4 s ± 611 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
